In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits import mplot3d

# Using library numpy-stl to handle STL files (https://pypi.python.org/pypi/numpy-stl)
from stl import mesh

# Observations

This problem sounds easy at first glance, but a number of edge cases make it tricky.
1. The "house" shape, which should need no support at all because the sloped roof holds up the top edge.
2. Anything touching the build plate, which doesn't need support.
3. A Cube, where the top edges don't need support, but the top face does.
4. A stalagtite, where the tip vertex needs support, but none of the edges or faces do.

# Plan

The basic approach I want to take is to determine whether a face/edge/vertex needs support purely by looking at the local properties of that feature and the things around it. We'll see if that works.

An alternative approach would be to build a directed graph indicating what things hold each other up, and reason about which edges need to be added to make it have a root.

An STL file contains information about faces, edges, and vertices.
## Faces
A face needs support when its is "flat" i.e. its normal vectors is close to the gravity vector. A cross product to get the normal vector, followed by a dot product with the gravity vector, makes computing that really easy.

Precomputation of $cos(\theta)$ should make the whole computation quite cheap.

## Edges
In addition to edges surrounding faces needing support, some edges themselves need to be supported, even if both of their neighboring faces are fine. An example is a horizontal edge connecting two faces in a "V" configuration. Notice that this edge requires support, while the inverse case (a peak) doesn't.

In this case, we are looking for an edge that is mostly horizontal (dot product again), and where neither (none) of its neighboring faces is sufficiently below it to support it (using their normals). Depending on what's next in the pipeline, it may or may not matter whether an edge adjacent to a supported face is supported or not.

This assumes that your FDM machine cannot perform bridging operations.

## Vertices

In addition to the cases above, a vertex needs support if none of its neighboring vertices are sufficiently below it. (i.e. tip of a [stalagtite](http://media.gettyimages.com/photos/stalactites-and-stalagmites-in-jenolan-caves-picture-id595906719?s=612x612) )

# Implementation

In [14]:
down = np.array([0, 0, 1])
theta = np.deg2rad(45)

epsilon = 0.000001 # A very small number, to deal with floating point rounding.

def isMostlyUp(vec):
    """
    Returns true if the provided 3-vector is pointning within theta of up
    """
    cosangle = np.dot(vec, down) / (np.linalg.norm(down) * np.linalg.norm(vec))
    costheta = np.cos(theta)
    
    return cosangle >= costheta

def isMostlyVertical(vec):
    """
    Returns true if the provided 3-vector is pointning within theta of up or down
    """
    cosangle = np.dot(vec, down) / (np.linalg.norm(down) * np.linalg.norm(vec))
    costheta = np.cos(theta)
    
    return cosangle >= costheta or cosangle <= -costheta

tests = [[0,0,1],
        [0,1,2],
        [0,2,1],
        [0,-1,-3],
        [0,4,0],
        ]

for t in tests:
    print t, isMostlyVertical(t)

[0, 0, 1] True
[0, 1, 2] True
[0, 2, 1] False
[0, -1, -3] True
[0, 4, 0] False


In [21]:
def almostEqual(v1, v2):
    """
    Checks whether two values (numbers, vectors, or arrays) are equal within epsilon.
    """
    return np.max(np.abs(v2 - v1)) <= epsilon

In [22]:
def isOnBuildPlate(point):
    # The build plate is assumed to be a infinite plane perpendicular 
    # to gravity, passing through the point (0, 0, 0)
    return almostEqual(np.dot(point, down), 0)

In [17]:
def faceNeedsSupport(face):
    """
    face is a list of three vertices
    """
    p1, p2, p3 = face
    normal = np.cross(p2-p1, p3-p1)
    
    if not isMostlyVertical(normal):
        # The face itself is vertical
        return False
    else:
        # The face is horizontal
        for p in face:
            if not isOnBuildPlate(p):
                # At least one vertex is not on the build plate
                return True
        # All vertices on build plate
        return False

In [28]:
def edgeNeedsSupport(endpoints, faces):
    """endpoints is list of two points, 
    faces is a list of arbitrarily many neighboring faces, given as triples of points"""
    
    p1, p2 = endpoints
    
    if isMostlyVertical(p2-p1):
        # Vertical edges are supported by their endpoint
        return False
    
    if isOnBuildPlate(p1) and isOnBuildPlate(p2):
        # This edge is supported by the build plate
        return False        
    
    # Check for a face that supports this edge
    for face in faces:
        # We need to check whether the face is "below" the edge.
        # Note that this can happen even if the third point on the edge is above 
        # both of the points on the edge.
        
        otherPoints = [p for p in face if (not almostEqual(p, p1) and not almostEqual(p, p2))]
        if len(otherPoints) != 1:
            raise Exception('Face provided not containing given edge')
            
        p3 = otherPoints[0]
        
        # The easiest way to check this is to which side of this edge the face is on
        # is to compute the cross product of this edge's vector with the vector to the third point,
        # as well as the cross product of this edge's vector and gravity. If those vectors are within 
        # +-90 degrees of each other, the face is below the edge.
        
        faceNormal = np.cross((p2-p1), (p3-p1))
        gravityNormal = np.cross((p2-p1), down)
        
#         faceNormal /= np.linalg.norm(faceNormal)
#         gravityNormal /= np.linalg.norm(gravityNormal)
        
        if np.dot(faceNormal, gravityNormal) > 0:
            # The face is "below" this edge
            if not faceNeedsSupport(face):
                # The face can support this edge
                return False
            
            
    return True
    

In [29]:
def vertexNeedsSupport(point, edges):
    """
    Takes as input:
        - A point Vector3
        - A list of edges connected to this point, given as pairs of points
    """
    
    if isOnBuildPlate(point):
        return False
    
    for edge in edges:
        otherPoints = [p for p in edge if not almostEqual(p, point)]
        if len(otherPoints) != 1:
            raise Exception('Edge provided not containing given vertex')
        other = otherPoints[0]
        
        # Check whether this point is mostly above the current edge
        if isMostlyUp(point - other):
            # This vertex is at the top of the provided edge, and so supported.
            return False
        
    return True


## STL library examples (reference)

In [ ]:
# Create a new plot
figure = plt.figure()
axes = mplot3d.Axes3D(figure)

# Load the STL files and add the vectors to the plot
your_mesh = mesh.Mesh.from_file('tests/cube.stl')
axes.add_collection3d(mplot3d.art3d.Poly3DCollection(your_mesh.vectors))

# Auto scale to the mesh size
scale = your_mesh.points.flatten(-1)
axes.auto_scale_xyz(scale, scale, scale)


In [ ]:
your_mesh.vectors